This code comes from https://nbviewer.org/github/afiliot/TPDUIA/blob/main/TPDUIA/2022/whole_slide_images.ipynb

In [1]:
# librairies utilitaires
import os
from pprint import pprint
from glob import glob
from random import sample

# numpy pour les matrics
import numpy as np

# visualisation de la progression des processus
from tqdm import tqdm
from p_tqdm import p_map

# manipulation et traitement des WSI
import openslide

# visualisation
from PIL import Image
import matplotlib.pyplot as plt
#from scripts import create_dzg, print_dz_properties
from typing import Dict, List, Tuple, Union


In [2]:
wsi_folder = '/media/user/Ligue_RNA_2partie/PCNSL_2022/0_ndpi'

In [3]:
wsi_paths = os.listdir(wsi_folder)


# Separation of tissue and white background


In [1]:
import cv2
from skimage.filters import threshold_multiotsu

def wsi_to_numpy(wsi: openslide.OpenSlide, level: int = 3) -> np.ndarray:
    """Transform a slide in .svs format to numpy matrix """
    assert level > 0, 'Select a lower resolution level!'
    image = np.asarray(
        wsi.read_region(
            (0, 0),
            level,
            wsi.level_dimensions[level]
        ).convert('RGB')
    )
    return image
def segment_region(
    wsi: openslide.OpenSlide,
    scale_factor: float = 1/64,
    classes: int = 3,
    smooth: bool = False,
    fill_holes: bool = False,
    plot: str = None
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Separation of tissue and white background by using Otsu method
    Parameters :
        scale_factor: float between 0 et 1 quantifying zoom level.
                      if scale_factor = 1, the slide with the higher resolution level is loaded into memory
        classes     : classes number in thresholding Otsu method
                      to see more details : https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_multiotsu.html
                      By default, 3 pixels classes are extracted from the slide 
        smooth      : smoothing of the segmentation mask (erosion then dilation). Useful to extract sparsely populated tissues
        fill_holes  : to fill holes from mask segmentation .
        plot        : display of segmentation steps 'all channels' ou 'lab only'
    """
    scale_factor = 1/wsi.level_downsamples[-1] if not scale_factor else scale_factor
    # choix du niveau de résolution le plus approprié en fonction du
    # zoom spécifié par l'utilisateur
    level = wsi.get_best_level_for_downsample(1/scale_factor)
    # conversion de la lame en matrice numpy
    img = wsi_to_numpy(wsi, level)
    # initialisation du masque de segmentation
    def get_mask(channel: str = 'lab') -> List[np.ndarray]:
        """Calculates a segmentation mask.
        If channel = 'lab', first converts the image to LAB space.
        Otherwise, use the red, green or blue channel to calculate the optimum
        thresholds for separating the different image layers."""
        # conversion du RGB au LAB
        # plus de détails ici : https://en.wikipedia.org/wiki/CIELAB_color_space
        # et ici pour la méthode de calcul:
        # https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_lab
        new_img = img.copy()
        mask = np.ones((img.shape[:2]))
        # application de la méthode d'Otsu sur l'image dans l'espace de couleurs LAB
        # ou sur les canaux Rouge, Vert ou Bleu de l'espace RGB.
        if channel == 'A (LAB)':
            lab = cv2.cvtColor(new_img, cv2.COLOR_BGR2LAB)[..., 1]
            _t = threshold_multiotsu(lab, classes)[0]
        elif channel == 'red (RGB)':
            lab = new_img[..., 0]
            _t = threshold_multiotsu(lab, classes)[1]
        elif channel == 'green (RGB)':
            lab = new_img[..., 1]
            _t = threshold_multiotsu(lab, classes)[1]
        elif channel == 'blue (RGB)':
            lab = new_img[..., 2]
            _t = threshold_multiotsu(lab, classes)[1]
        # définition du masque de segmentation
        if channel == 'A (LAB)':
            mask = 1-(lab < _t)*1
        else:
            mask = (lab < _t)*1
            lab = 1 - lab
        # les pixels du fond sont codés en RGB comme (255, 255, 255)
        new_img[np.where(mask == 0)] = 255
        if smooth:
            mask = binary_closing(mask, iterations=15)
            mask = binary_opening(mask, iterations=10)
            if fill_holes:
                mask = binary_fill_holes(mask)
            new_img[np.where(mask == 0)] = 255
        return lab, mask, new_img
    # affichage des segmentations basées sur LAB, canal ROUGE, VERT puis BLEU (4x4 images)
    if plot == 'all channels':
        _, axes = plt.subplots(nrows=4, ncols=4, figsize=(27, 24))
        mag = int(wsi.properties['openslide.objective-power']) / wsi.level_downsamples[level]
        for i, channel in enumerate(['A (LAB)', 'red (RGB)', 'green (RGB)', 'blue (RGB)']):
            # on calcule le masque de segmentation sur la base de 4 canaux différents.
            lab, mask, new_img = get_mask(channel)
            axes[i, 0].imshow(img); axes[i, 0].set_axis_off()
            axes[i, 0].set_title("Level %d (size. %.3f) from native image" % (level, mag))
            axes[i, 1].imshow(lab); axes[i, 1].set_axis_off()
            axes[i, 1].set_title(f"channel {channel}")
            axes[i, 2].imshow(mask, cmap='gray'); axes[i, 2].set_axis_off()
            axes[i, 2].set_title('Segmentation mask')
            axes[i, 3].imshow(img); axes[i, 3].set_axis_off()
            axes[i, 3].imshow(mask, alpha=0.3, cmap='gray')
            axes[i, 3].set_title('Tissue selection')
        plt.show()
        return None
    # affichage de la segmentation basée sur LAB uniquement (1x4 images)
    else:
        lab, mask, new_img = get_mask(channel='A (LAB)')
        if plot == 'lab only':
            _, axes = plt.subplots(nrows=1, ncols=4, figsize=(27, 6))
            mag = int(wsi.properties['openslide.objective-power']) / wsi.level_downsamples[level]
            # on calcule le masque de segmentation sur la base du canal A de LAB.
            lab, mask, new_img = get_mask(channel='A (LAB)')
            axes[0].imshow(img); axes[0].set_axis_off()
            axes[0].set_title("Level %d (size. %.3f) from native image" % (level, mag))
            axes[1].imshow(lab); axes[1].set_axis_off()
            axes[1].set_title(f"Channel A (LAB)")
            axes[2].imshow(mask, cmap='gray'); axes[2].set_axis_off()
            axes[2].set_title('Segmentation mask')
            axes[3].imshow(img); axes[3].set_axis_off()
            axes[3].imshow(mask, alpha=0.3, cmap='gray')
            axes[3].set_title('Tissue selection')
            plt.show()
        return img, lab, new_img, mask

_ = segment_region(wsi, plot='all channels')

# Patches creation

In [8]:
from openslide.deepzoom import DeepZoomGenerator as DZG

def create_dzg(
    wsi: openslide.OpenSlide,
    wsi_path: str,
    tile_size: int,
    target_shape: int,
    overlap: int,
    thresh: float,
    scale_factor: float = 1/32,
    patch_precomputed: bool = True
) -> DZG:
    """Créée un objet DZG avec des attributs supplémentaires utiles
    pour le traitement de la lame et des patches.
    
    Paramètres:
        wsi               : slide in openslide.OpenSlide format
        wsi_path          : path to slide 
        tile_size         : patches size in micrometers 
        target_shape      : saved patches size in pixels 
        overlap           : percentage of overlapping patches 
        thresh            : maximal percentage of white background that can contains a patch
        scale_factor      : zoom out percentage (by default=1/32)
        patch_precomputed : if patches have been created, it's the directory 
                            PATCHES_BACKUP (patch_precomputed=True).
                            else (FALSE), patches will be created in PATCHES directory to avoid overwriting data
                       
    Return:
        dz : DeepZoomGenerator object with supllementary attributes 
    """

    ppx = float(wsi.properties['openslide.mpp-x']) # micrometers per pixels
    tile_size_ = round(tile_size / ppx)
    overlap_ = round(tile_size_ * overlap)
    # creation of DGZ object
    dz = DZG(
        osr=wsi,
        tile_size=tile_size_,
        overlap=overlap_
    )
    # on exécute la segmentation pour récupérer le masque
    dz.img_raw, dz.lab, dz.img_new, dz.mask = segment_region(wsi, scale_factor=scale_factor, classes=3)
    # on enregistre la lame
    dz.wsi = wsi
    dz.wsi_path = wsi_path
    # tile_size est la taille des patchs en micromètres
    dz.tile_size = tile_size
    # tile_size est la taille des patchs en micromètres
    # tile_size_ est la taille des patchs en pixels
    dz.tile_size = tile_size
    dz.tile_size_ = tile_size_
    # target_shape est la taille des patchs en pixels
    # et sera la taille finale des patches une fois
    # sauvegardés
    dz.target_shape = target_shape
    # overlap_ est la taille de l'overlap en pixels
    dz.overlap = overlap
    dz.overlap_ = overlap_
    # on ajoute le seuil maximal de fond blanc que peut
    # contenir un patch
    dz.thresh = thresh
    # w_0 et h_0 sont les dimensions (largeur, hauteur)
    # de la lame au niveau de résolution le plus élevé
    dz.w_0, dz.h_0 = dz.level_dimensions[-1]
    # w_m et h_m sont les dimensions (largeur, hauteur)
    # du masque de segmentation au niveau de résolution
    # plus faible
    dz.w_m, dz.h_m = dz.mask.shape[::-1]
    # f_w et f_h sont les taux exacts de dezoom et sont 
    # calculés comme les ratios de w_0 sur w_m et 
    # h_0 sur h_m respectivement
    dz.f_w, dz.f_h = dz.w_0 / dz.w_m, dz.h_0 / dz.h_m
    # ce paramètre spécifie l'indice du plus haut niveau
    # de résolution de l'objet DZG
    dz.l0_z = dz.level_count - 1
    # ainsi que l'identifiant du patient
    dz.patient_id = os.path.split(wsi_path)[0][-4:]
    # on spécifie le dossier où seront stockées les patches
    # si les patches ont déjà été créés, il s'agit du dossier
    # PATCHES_BACKUP
    # sinon, les patches seront créés dans le dossier PATCHES
    # pour ne pas écraser les données!
    if not patch_precomputed:
        dz.output_folder = f'{PATH}/'#PATCHES/{dz.patient_id}_taille-{tile_size}/'
    else:
        dz.output_folder = f'{PATH}/PATCHES_BACKUP/{dz.patient_id}_taille-{tile_size}/'
    # on créée ce dossier s'il n'existait pas encore
    os.makedirs(dz.output_folder, exist_ok=True)
    # on ajoute le niveau de dézoom par niveau de résolution
    # de l'objet DZG
    dz.level_downsamples = [
        round(
            dz.level_dimensions[-1][0] / dzl[0], 0
        ) for dzl in dz.level_dimensions
    ]
    # on ajoute le nombre de patches par niveau de résolution
    # de l'objet DZG
    dz.level_tile_counts = [
        dzl[0] * dzl[1] for dzl in dz.level_tiles
    ]
    return dz

def print_dz_properties(dz: DZG):
    print(f'Nombre de niveaux de résolution\n{dz.level_count}\n')
    print(f'Nombre total de patchs\n{dz.tile_count}\n')
    print(f'Nombre de patchs par niveau de résolution\n{dz.level_tile_counts[::-1]}\n')
    print(f'Nombre de patchs par niveau de résolution selon (largeur, longueur)\n{dz.level_tiles[::-1]}\n')
    print(f'Dimension des niveaux de résolution\n{dz.level_dimensions[::-1]}\n')
    print(f'Niveau de dézoom par niveau de résolution\n{dz.level_downsamples[::-1]}\n')

In [9]:
import itertools

def get_grid(dz: DZG) -> List[Tuple[int, int]]:
    """Retourne une liste de coordonnées (sur la largeur et la longueur)
    de toutes les tiles du plus haut niveau de résolution de la lame.
    Par exemple, une valeur possible de list(grid) est:
    [(0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]"""
    dz.w_tiles, dz.h_tiles = dz.level_tiles[dz.l0_z]
    grid = list(itertools.product(range(dz.h_tiles), range(dz.w_tiles)))
    assert len(grid) == dz.h_tiles * dz.w_tiles
    return grid

In [10]:
def get_regions(dz: DZG, i: int, j: int) -> Tuple[List[int], List[int]]:
    """Retourne une région d'intérêt étant donné deux indices i et j.
    Paramètres:
        i   : indice selon la largeur de la lame
        j   : indice selon la hauteur de la lame

    Outputs:
        region_0 :  liste de 4 entiers (w_0, h_0, t_w, t_h) avec w_0 et h_0
                    les coordonnées du coin en haut à gauche du patch d'indice
                    (i, j). t_w et t_h sont la largeur et la hauteur du patch
                    en pixel. En principe, t_w = t_h sauf pour les patches 
                    présents sur les bordures de la lame. Les coordonnées w_0,
                    h_0, t_w, t_h sont prises pour le plus haut niveau de 
                    résolution.
        
        region_l :  liste de 4 entiers (h_start, h_end, w_start, w_end) 
                    correpondant aux coordonnées du patch équivalent à
                    un niveau de résolution plus faible (ex. 32). Ces
                    coordonnées permettront de calculer le pourcentage 
                    de fond blanc sur le patch grâce au masque de segmen-
                    tation.
    """
    # coordonnées dans le niveau de résolution le plus élevé
    (w_0_ij, h_0_ij), _, (d_w_0_ij, d_h_0_ij) = dz.get_tile_coordinates(dz.l0_z, (j, i))
    # coordonnées dans le niveau de résolution le plus faible
    d_h_l_ij, d_w_l_ij = round(d_h_0_ij / dz.f_h), round(d_w_0_ij / dz.f_w)
    start_w_l_ij, start_h_l_ij = round(w_0_ij / dz.f_w), round(h_0_ij / dz.f_h)
    region_0 = [w_0_ij, h_0_ij, d_h_0_ij, d_w_0_ij]
    region_l = [start_h_l_ij, start_h_l_ij + d_h_l_ij, start_w_l_ij, start_w_l_ij + d_w_l_ij]
    return region_0, region_l

In [11]:
from PIL.Image import BILINEAR
def ij_tiling(
        dz: DZG,
        coords: Tuple[int],
        plot: bool = False,
        save: bool = True
) -> Union[None, Tuple[np.ndarray, str]]:
    """Lit et stocke les patches de la lame selon des
    coordonnées (i, j) spécifiées en paramètres `coords` si
    le patch en question dispose d'un niveau de fond blanc inférieur
    à celui précisé précédemment (`dz.thresh`).
    """
    i, j = coords
    region_0, region_l = get_regions(dz, i, j)
    # lecture du masque binaire background / foreground au niveau
    # de résolution plus faible. On réutilise les coordonnées 
    # calculées avec la fonction "get_regions" !
    mask_l_ij = dz.mask[region_l[0]:region_l[1], region_l[2]:region_l[3]]
    # pourcentage de background (fond blanc)
    bgp = 1 - mask_l_ij.mean()
    # si le patch contient suffisamment de tissu, on lit puis stocke
    # le patch correspondant au niveau de résolution le plus élevé
    if bgp < dz.thresh:
        # lecture de la région d'intérêt et conversion en RGB
        tile_0_ij_ = dz.get_tile(dz.l0_z, (j, i)).convert('RGB')
        size_0_ij = tile_0_ij_.size
        if size_0_ij[0] == size_0_ij[1]:
            # re-dimensionnement selon une taille cible "target_shape"
            tile_0_ij = tile_0_ij_.resize(
                (dz.target_shape, dz.target_shape), BILINEAR
            )
        else:
            tile_0_ij = copy(tile_0_ij_)
        # nom de fichier
        tile_path = 'tilesize%d_i%d_j%d_w%d_h%d_dw%d_dh%d_overlap%f_pbg%f' % (
                tuple([dz.tile_size, i, j] + region_0 + [dz.overlap, bgp])
            )
        if save:
            # stockage du patch avec les informations utiles dans le
            # nom de fichier
            tile_path = os.path.join(dz.output_folder, tile_path + '.jpg')
            tile_0_ij.save(tile_path)
        # rendu graphique si plot = True
        if plot:
            # conversion de l'image en matrice numpy
            tile_0_ij = np.array(tile_0_ij)
            tile_0_ij_ = np.array(tile_0_ij_)
            # on créée la figure et les axes
            fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(17, 3))
            
            # 0: patch au niveau de résolution le plus élevé avant redimensionnement
            # en target_shape x target_shape pixels
            axes[0].imshow(tile_0_ij_)
            axes[0].set_title(
                f'% fond blanc {round(bgp, 2)}\n(dezoom {round(dz.f_h, 0)})\n({dz.tile_size}μm x {dz.tile_size}μm)\n({dz.tile_size_}p x {dz.tile_size_})'
            )
            
            # 1: patch au niveau de résolution le plus élevé après redimensionnement
            axes[1].imshow(tile_0_ij)
            axes[1].set_title(
                f'% fond blanc {round(bgp, 2)}\n(dezoom {round(dz.f_h, 0)})\n({dz.tile_size}μm x {dz.tile_size}μm)\n({dz.target_shape}p x {dz.target_shape}p)'
            )

            # 2: masque de segmentation pour le patch considéré
            axes[2].imshow(mask_l_ij, cmap='gray')
            axes[2].set_title(
                f'% fond blanc {round(bgp, 2)}\n(dezoom {round(dz.f_h, 0)})\n({mask_l_ij.shape[0]}p x {mask_l_ij.shape[1]}p)'
            )
            
            # 3: redimensionnement du masque et superposition au patch
            # affichage de la classe "fond blanc"
            mask_l_ij_resized = resize(mask_l_ij, (tile_0_ij.shape[:-1]))
            axes[3].imshow(tile_0_ij)
            axes[3].imshow(-mask_l_ij_resized, alpha=0.5, cmap='gray')
            axes[3].set_title(f'Background\n(dezoom 1.0)\n({tile_0_ij.shape[0]}p x {tile_0_ij.shape[1]}p)')
            
            # 4: idem
            # # affichage de la classe "Tissus"
            axes[4].imshow(tile_0_ij)
            axes[4].imshow(mask_l_ij_resized, alpha=0.5, cmap='gray')
            axes[4].set_title(f'Tissus\n(dezoom 1.0)\n({tile_0_ij.shape[0]}p x {tile_0_ij.shape[1]}p)')
            plt.show()
        return tile_0_ij, tile_path
    return None, None


/tmp/ipykernel_9682/2422214261.py:1: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  from PIL.Image import BILINEAR


In [12]:
from skimage.transform import resize


In [13]:

def plot_tiles_on_row(
        dz: DZG,
        grid: List[Tuple[int, int]],
        j: int = 200,
        n_patches: int = None
    ) -> None:
    # on récupère toutes les coordonnées de la ligne j
    row = [g for g in grid if g[0] == j]
    n_patches = len(row) if n_patches is None else n_patches
    tiles = []
    # on copie l'objet dz et on déclare un seuil de 1.01,
    # ce qui a pour effet de sélectionner toutes les patchs
    # de la ligne j
    dz_ = copy(dz);dz_.thresh = 1.01
    for coords in tqdm(row, desc=f'Extracting tiles on row {j}...'):
        out = ij_tiling(dz_, coords, plot=False)
        if out is not None:
            tile = out[0]
            tiles.append(tile)
    # on affiche les 100 premiers patches de la liste
    n = len(tiles)
    fig, axes = plt.subplots(n_patches//10, 10, figsize=(14, (n_patches//10)*(16/10)))
    for i, tile in enumerate(tiles[:n_patches]):
        ax = axes[i//10, i%10]
        ax.imshow(tile)
        ax.set_title(f'({j}, {i})')
        ax.axis('off')

def visualize_band(
    wsi: openslide.OpenSlide,
    dz: DZG,
    row: int = 200, delta: int = 200, band_level: int = 1
) -> None:
    """Paramètres
        row        : ligne d'intérêt à visualiser
        delta      : delta sert à ajuster la zone à visualiser (arbitraire)
        band_level : niveau de résolution de la bande à visualiser (défaut x20)"""

    wsi_width, wsi_height = wsi.dimensions # largeur et longueur de la lame
    tiles_on_width, tiles_on_height = dz.level_tiles[-1] # n patchs sur largeur et longueur
    # position sur la largeur du coin en haut à gauche
    band_w0 = 0
    # position sur la hauteur du coin en haut à gauche
    band_h0 = int(row/tiles_on_height * wsi_height) - delta
    # largeur de la bande
    band_width = 256 * 10
    # hauteur
    band_height = 256
    display_region(wsi, band_w0, band_h0, band_level, band_width, band_height)
    

In [14]:
from copy import copy

In [15]:
def display_region(
    wsi: openslide.OpenSlide,
    x: int, y: int, level: int, width: int, height: int
) -> None:
    """Sélectionne la région d'intérêt sur la lame et affiche les informations
    correspondantes.
    Paramètres:
        x, y   : origine en haut à gauche selon la largeur et la hauteur
        level  : niveau de résolution (0 le plus élevé)
        width  : largeur de la région
        height : hauteur de la région
    """
    region = get_region(wsi, x, y, level, width, height)
    physical_width, physical_height = get_dimensions(wsi, level, width, height)
    print(f"""
* Dimensions numériques (pixels)      : {width}p x {height}p ({format(width * height, ',d').replace(',',' ')} pixels)
* Dimensions physiques (cm ou μm)     : {physical_width} x {physical_height}
* Taille décompressée (Mo)            : {round(width*height*24/1e6, 2)}
    """)
    display(region)

In [16]:
def get_region(
    wsi: openslide.OpenSlide,
    x: int, y: int, level: int, width: int, height: int
) -> Image:
    """Sélectionne la région d'intérêt sur la lame.
    Paramètres:
        x, y   : origine en haut à gauche selon la largeur et la hauteur
        level  : niveau de résolution (0 le plus élevé)
        width  : largeur de la région
        height : hauteur de la région
    """
    region = wsi.read_region((x,y), level, (width, height))
    return region

In [17]:
def get_regions(dz: DZG, i: int, j: int) -> Tuple[List[int], List[int]]:
    """Retourne une région d'intérêt étant donné deux indices i et j.
    Paramètres:
        i   : indice selon la largeur de la lame
        j   : indice selon la hauteur de la lame

    Outputs:
        region_0 :  liste de 4 entiers (w_0, h_0, t_w, t_h) avec w_0 et h_0
                    les coordonnées du coin en haut à gauche du patch d'indice
                    (i, j). t_w et t_h sont la largeur et la hauteur du patch
                    en pixel. En principe, t_w = t_h sauf pour les patches 
                    présents sur les bordures de la lame. Les coordonnées w_0,
                    h_0, t_w, t_h sont prises pour le plus haut niveau de 
                    résolution.
        
        region_l :  liste de 4 entiers (h_start, h_end, w_start, w_end) 
                    correpondant aux coordonnées du patch équivalent à
                    un niveau de résolution plus faible (ex. 32). Ces
                    coordonnées permettront de calculer le pourcentage 
                    de fond blanc sur le patch grâce au masque de segmen-
                    tation.
    """
    # coordonnées dans le niveau de résolution le plus élevé
    (w_0_ij, h_0_ij), _, (d_w_0_ij, d_h_0_ij) = dz.get_tile_coordinates(dz.l0_z, (j, i))
    # coordonnées dans le niveau de résolution le plus faible
    d_h_l_ij, d_w_l_ij = round(d_h_0_ij / dz.f_h), round(d_w_0_ij / dz.f_w)
    start_w_l_ij, start_h_l_ij = round(w_0_ij / dz.f_w), round(h_0_ij / dz.f_h)
    region_0 = [w_0_ij, h_0_ij, d_h_0_ij, d_w_0_ij]
    region_l = [start_h_l_ij, start_h_l_ij + d_h_l_ij, start_w_l_ij, start_w_l_ij + d_w_l_ij]
    return region_0, region_l

def get_dimensions(
    wsi: openslide.OpenSlide,
    level: int = 0,
    width: int = 512,
    height: int = 512
) -> Tuple[str, str]:
    """Calcule la dimension physique d'une région en micromètres ou centimètres."""
    physical_width = width * wsi.level_downsamples[level] * float(wsi.properties['openslide.mpp-x'])
    if physical_width / 10000 > 0.1:
        physical_width /= 10000
        physical_width = round(physical_width, 3)
        physical_width = str(physical_width)+'cm'
    else:
        physical_width = round(physical_width, 3)
        physical_width = str(physical_width)+'μm'
    physical_height = height * wsi.level_downsamples[level] * float(wsi.properties['openslide.mpp-y'])
    if physical_height / 10000 > 0.1:
        physical_height /= 10000
        physical_height = round(physical_height, 3)
        physical_height = str(physical_height)+'cm'
    else:
        physical_height = round(physical_height, 3)
        physical_height = str(physical_height)+'μm'
    return (physical_width, physical_height)

In [18]:
import matplotlib.patches as patches

def plot_patch_mask(PATH_output, dz: DZG, figsize: Tuple[int, int] = (20, 10)) -> None:
    """Affichage de la mosaïque de patches sur la lame."""
    # on récupère l'ensemble des données d'intérêt pour
    # l'affichage graphique: l'objet openslide, l'image
    # de la lame (qui rentre en mémoire!), et le masque
    # de segmentation
    wsi, image, mask = dz.wsi, dz.img_raw, dz.mask
    # on récupère également les chemins des patches
    # qui ont été créés précédemment
    tiles_names = os.listdir(dz.output_folder)
    # on calcule les dimensions de l'image (ce sont
    # les mêmes que celles du masque)
    thumb_h, thumb_w = image.shape[:2]
    # on calcule le facteur de dezoom pour l'affichage
    # de l'image finale
    dwn_w = wsi.level_dimensions[0][0] / thumb_w
    dwn_h = wsi.level_dimensions[0][1] / thumb_h
    # on initialise l'image où se superposeront les patches,
    # avec les mêmes dimensions que l'image d'origine
    img = Image.new('1', (thumb_h, thumb_w), 0)
    # on créée une figure vierge
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    ax = axes[1]
    # on itère sur tous les patchs et on créée à chaque fois un rectangle
    # qui délimite le patch en question (s'il a été sélectionné par l'algorithme!,
    # c'est-à-dire si sa proportion de tissus est suffisant)
    for idx, tile in tqdm(enumerate(tiles_names), total=len(tiles_names)):
        # on récupère les coordonnées du patch (niveau de résolution le 
        # plus élevé),
        # que l'on divisent ensuite par le niveau de dezoom pour convertir
        # ces coordonnées dans le niveau de résolution plus faible 
        t_s = tile.split('_')
        h_t = int(t_s[-5][1:]) / dwn_h
        w_t = int(t_s[-6][1:]) / dwn_w
        dh_t = int(t_s[-3][2:]) / dwn_h
        dw_t = int(t_s[-4][2:]) / dwn_w
        # on génère un dessin de rectangle avec les coordonnées re-dimensionnées
        # que l'on ajoute à l'image vierge `img`
        patch = patches.Rectangle((w_t, h_t), dh_t, dw_t, alpha=0.5, ec='blue')
        ax.add_patch(patch)
        ImageDraw.Draw(img).rectangle(
            [w_t, h_t, dh_t + w_t, dw_t + h_t],
            outline=1, fill=1
        )
    # on convertit l'image avec les rectangles en une matrice numpy
    # faite de 0 et 1
    tiles_mask = np.array(img) * 1.0
    # on l'affiche avec l'image brute chargée au début de la fonction
    ax.imshow(image, alpha=0.7, origin='upper');ax.axis('off')
    ax.set_title('Lame avec les patchs sélectionnés')
    ax = axes[0]
    ax.imshow(image, origin='upper');ax.axis('off')
    ax.set_title('Lame brute')
    fig.savefig(PATH_output+'mask_tiles.jpg')


In [19]:
from PIL import Image, ImageDraw
#plot_patch_mask(dz)

# For all ndpis 

In [20]:
PATH_wsi = '/media/user/Ligue_RNA_2partie/PCNSL_2022/0_ndpi/'

In [21]:
patients_ndpi = os.listdir(PATH_wsi)
len(patients_ndpi)

106

In [26]:
tile_size = 224*0.46 #micromètres
target_shape = 224 #pixels
overlap = 0
threshold_background_max = 0.05

In [2]:
from copy import copy
for i in patients_ndpi :
    patient = i[:-5]
    print(patient)
    wsi_path = PATH_wsi+i
    wsi = openslide.OpenSlide(wsi_path)
    PATH = "/media/user/Ligue_RNA_2partie/PCNSL_2022/1_patches_filiot_224/"+patient
    if not os.path.exists("/media/user/Ligue_RNA_2partie/PCNSL_2022/1_patches_filiot_224/"+patient) :
        os.makedirs("/media/user/Ligue_RNA_2partie/PCNSL_2022/1_patches_filiot_224/"+patient)
        print("dossier créé")
    if not os.path.exists('/media/user/Ligue_RNA_2partie/PCNSL_2022/1_mask_tiles_224/'+patient+'__mask_tiles.jpg') :

        dz = create_dzg(
            wsi, wsi_path, tile_size, target_shape, overlap, threshold_background_max,
            patch_precomputed=False
        )
        print('créé')
        print(dz.output_folder)
        tiles_names = os.listdir(dz.output_folder)
        grid = get_grid(dz)
        print('OK')
        _ = p_map(lambda x: ij_tiling(dz, x, plot=False), grid)
        print(f'Done. Stored {len(os.listdir(dz.output_folder))} patches.')
        plot_patch_mask(PATH, dz)
    else:
        print(patient, "déjà segmenté")

In [ ]:
plot_patch_mask(PATH, dz)